In [ ]:
6 tries left
2 karun
4 haoyu


In [45]:
import pandas as pd
import sys
from tqdm import tqdm
from gensim.models import Word2Vec
import random
import multiprocessing 
import pickle

In [46]:
READ = "r"
# stores the mapping from a person to the people they follow
follows_original = {}
follows = {}
unique_ids = set({})
follow_counts = {}
# stores deleted edges from the training data
dev_list = []
positive_edges_list = []
negative_edges_list = []

# --------- Hyper-parameters ----------- #
RANDOM_WALK_LENGTH = 10
WALKS_PER_NODE = 10000
# -------------------------------------- #

# TODO change the path
TRAINING_FILE_PATH = '../../Data/train.txt'
#MODEL_PATH = '../data/rw.model'
#WORD_VECTORS_PATH = '../data/word_vectors'
#WORD_VECTOR_MATRIX = "../data/rw3"

In [47]:
# load the follows data-structure
def load_training_data():
    train_data = open(TRAINING_FILE_PATH, READ)
    for entry in train_data:
        ids = entry.strip().split('\t')
        # add all the entries to the SET to find the unique ids in the training data
        unique_ids.update(ids)

        # extract the source
        source = ids[0]
        # extract the people who the user follows, by removing the user-id from the ids
        sinks = set(ids[1:])
        print
        
        # follows original 
        follows_original[source] = sinks
        
        # creating deleted edges from training data
        if len(sinks) >= 2:
            del_sink = random.choice(tuple(sinks))
            positive_edges_list.append((source,del_sink,1))
            negative_edges_list.append((source, random.choice(range(500)),0))
            # removing the sink nodes from training data
            sinks = sinks - set(del_sink)
            
        
        # maps user to the people they follow
        follows[source] = sinks

        # keep a map of no of people they follow
        follow_counts[source] = len(sinks)
        
    #df = pd.DataFrame({})
    dev_list.extend(positive_edges_list[:5000])
    dev_list.extend(negative_edges_list[:5000])
    
    with open('pickle_files/follows.pickle', 'wb') as handle:
        pickle.dump(follows, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #dataframe dev data
    df_gt = pd.DataFrame(dev_list,columns=["Source","Sink","State"])
    df = df_gt[df_gt.columns.difference(["State"])]
    
    with open('pickle_files/df.pickle', 'wb') as handle:
        pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('pickle_files/df_gt.pickle', 'wb') as handle:
        pickle.dump(df_gt, handle, protocol=pickle.HIGHEST_PROTOCOL)

load_training_data()

In [43]:
# creating negative samples for dev set

#negative_edges_list =[]
#for i in range(5000):
#item = random.sample(follows_original.keys(),5000)
#print(len(item))
#print(item)
#for i in item:
    #n_item = tuple(unique_ids - follows_original[i])
    #n_node = random.choice(n_item)
    #negative_edges_list.append((i,n_node,"False"))
#print(negative_edges_list[:10])

In [44]:
#df = pd.DataFrame({})
#dev_list.extend(positive_edges_list[:5000])
#dev_list.extend(negative_edges_list)
#dataframe dev data
#df_gt = pd.DataFrame(dev_list,columns=["Source","Sink","State"])
#df = df_gt[df_gt.columns.difference(["State"])]
#print(len(df))
#print(len(df_gt))
#with open('pickle_files/df.pickle', 'wb') as handle:
    #pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('pickle_files/df_gt.pickle', 'wb') as handle:
    #pickle.dump(df_gt, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# performs a single random walk on the passed node for the given number of hops
# please note that this walk is ALWAYS from a source to a sink
def perform_random_walk(node):
    
    # stores the path of a single random walk
    random_walk = [node]
    hops = RANDOM_WALK_LENGTH
    while hops >= 0:
        hops -= 1
        if node in follows:
            neighbours = list(follows[node])
            # remove the neighbours already visited
            #neighbours = list(set(neighbours) - set(random_walk))
            if len(neighbours) == 0:
                break
        else:
            # cannot go any further
            break

        # choose one of the neighbouring nodes
        random_node = random.choice(neighbours)
        random_walk.append(random_node)
        # jump to the new node
        node = random_node

    return random_walk



In [ ]:
# returns an array of random walks
# they all are NOT of the same length
def get_random_walks():
    random_walks = []
    # perform random walks for all nodes who follow somebody
    for unique_node in tqdm(unique_nodes):
        if unique_node in follows:
            for index in range(min(WALKS_PER_NODE, follow_counts[unique_node])*4):
                random_walks.append(perform_random_walk(unique_node))
    #with open('../data/random_walks_4x.pickle', 'wb') as handle:
    #    pickle.dump(random_walks, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    return random_walks

In [ ]:
def create_and_save_model(random_walks):
    '''
    model = Word2Vec(size=300, workers = 5, alpha=0.03, window=4, sg=1, hs=0,
                     negative=10, ns_exponent = 0.75, min_alpha=0.0005,
                     seed=99)
    '''
    model = Word2Vec(size=400, workers = 5, alpha=0.025, window=3, sg=1, hs=1,
                     negative=10, min_alpha=0.0005,
                     seed=99)
    
    model.build_vocab(random_walks)
    model.train(random_walks, total_examples=model.corpus_count, epochs=10)
    model.save(MODEL_PATH)
    model.init_sims(replace=True)

    model.wv.save_word2vec_format(WORD_VECTOR_MATRIX)
    model.wv.save(WORD_VECTORS_PATH)

    # perform some sanity checks
    print(model.similar_by_word('540762'))
    print(model.wv.most_similar(positive=["540762"]))
    print(model.wv.similarity("540762", '2345899'))



In [ ]:
load_training_data()

In [ ]:
unique_nodes = list(unique_ids)

In [ ]:
random_walks = get_random_walks()
print(len(random_walks))

In [ ]:
follows = {}
unique_ids = set({})
follow_counts = {}

In [ ]:
with open('../data/random_walks_4x.pickle', 'rb') as handle:
    random_walks = pickle.load(handle)

In [ ]:
length = len(random_walks)
for i in range

In [ ]:
create_and_save_model(random_walks)

In [ ]:
from gensim.models import Word2Vec
import sys

TRAINING_FILE_PATH = '../data/train.txt'
TESTING_FILE_PATH = '../data/test-public.txt'
READ = "r"
WRITE = "w"
OUT_FILE_PATH = 'current_best.csv'

outfile = open(OUT_FILE_PATH, WRITE)

# Load the word2vec model
model = Word2Vec.load("../data/rw.model")
train_data = open(TRAINING_FILE_PATH, READ)

# None marker, indicates that processing did not happen
NULL = -5

outfile.write('Id,Predicted' + '\n')

sink2sources = {}
source2sinks = {}
source2sink_counts = {}


# compares the similarity of source,
def get_source_as_sink_similarity(source, sink):

    connected_sinks = source2sinks[source]

    c_sinks = 0
    total_sim = 0

    for connected_sink in connected_sinks:
        if connected_sink in model.wv.vocab:
            c_sinks += 1
            # print('-----', model.wv.similarity(connected_sink, sink))
            total_sim = total_sim + model.wv.similarity(connected_sink, sink)

    # print(c_sinks)

    if c_sinks > 0:
        return total_sim/c_sinks
    else:
        return -5


def get_sink_as_source_similarity(source, sink):

    connected_sources = sink2sources[sink]

    c_sources = 0
    total_sim = 0

    for connected_source in connected_sources:
        if connected_source in model.wv.vocab:
            c_sources += 1
            total_sim = total_sim + model.wv.similarity(connected_source, source)

    # print(c_sinks)

    if c_sources > 0:
        return total_sim/c_sources
    else:
        return NULL


# returns a representation for source
def get_representation_for_source(src):

    if src in model.wv.vocab:
        word_vector_rep = model.wv[src]
        # print(word_vector_rep)
    elif src in source2sinks and len(source2sinks[src]) > 0:
        # if the source is missing, but the source has some sinks in the train set
        # we then say that a source is defined by the avg of the sources which follows its sinks
        # need to weigh this in future !
        print('No Match', src)
        print(len(source2sinks[src]))

        connected_sinks = source2sinks[src]
        c_sinks = 0
        for sink in connected_sinks:
            if sink in model.wv.vocab:
                c_sinks += 1

        print(c_sinks)
    else:
        print('NOTHING', src)


# returns a rep for sink
def get_representation_for_sink(snk):
    if snk in model.wv.vocab:
        word_vector_rep = model.wv[snk]
        # print(word_vector_rep)
    else:
        print('No word vector', snk)


# loads the source to sinks and sink to sources map
def load_mappings():
    for entry in train_data:
        ids = entry.strip().split('\t')
        source = ids[0].strip()
        sinks = set(ids[1:])
        source2sinks[source] = sinks
        source2sink_counts[source] = len(sinks)
        for sink in sinks:
            if sink in sink2sources:
                sink2sources[sink.strip()].append(source)
            else:
                sink2sources[sink.strip()] = [source]


def predict_with_test_data():

    test_data = open(TESTING_FILE_PATH, READ)
    test_data.readline()
    no_match = 0

    index = 0

    for line in tqdm(test_data):
        index += 1
        data = line.split('\t')
        source, sink = data[1].strip(), data[2].strip()
        direct_sim = NULL
        source_as_sink_sim = NULL
        sink_as_source_sim = NULL
        total = 1000002

        # if both source and sink have word vector representations
        if source in model.wv.vocab and sink in model.wv.vocab:
            direct_sim = model.wv.similarity(source, sink)

        # transform source as a fn of the sinks it follows and then calculate the similarity between the two
        if sink in model.wv.vocab:
            source_as_sink_sim = get_source_as_sink_similarity(source, sink)

        if source in model.wv.vocab:
            sink_as_source_sim = get_sink_as_source_similarity(source, sink)

        if direct_sim == NULL and source_as_sink_sim == NULL and sink_as_source_sim == NULL:
            no_match += 1

        if direct_sim == NULL:
            total = total - 1
        if source_as_sink_sim == NULL:
            total = total - 1
        if sink_as_source_sim == NULL:
            print('error')
            #sys.exit(0)
            total = total - 1000000

        total_probability = max(0, direct_sim) + max(0, source_as_sink_sim) + max(0, sink_as_source_sim) * 1000000
        # total_probability = max(0, sink_as_source_sim) * 1000
        
        if total == 0:
            total_probability = 0.1
        else:
            total_probability = total_probability / total

        print(direct_sim, ' : ', source_as_sink_sim, ' : ', sink_as_source_sim, ' : ', total_probability)
        outfile.write(str(index) + ',' + str(total_probability) + '\n')

    print(no_match)


load_mappings()
predict_with_test_data()
outfile.close()

